### Reference

[Q Datatypes](https://code.kx.com/q/basics/datatypes/)

[qsql](https://code.kx.com/q/basics/qsql/)

[parquet](https://code.kx.com/kdb-x/modules/parquet/introduction.html) API

Operators:
- [avg](https://code.kx.com/q/ref/avg/#avg)
- [wavg](https://code.kx.com/q/ref/avg/#wavg)
- [xbar](https://code.kx.com/q/ref/xbar/)
- [$](https://code.kx.com/q/ref/cast/) - Datatype casting
- [asc](https://code.kx.com/q/ref/asc/)
- [desc](https://code.kx.com/q/ref/desc/)
- [!](https://code.kx.com/q/ref/enkey/#unkey) - Unkey
- [deltas](https://code.kx.com/q/ref/deltas/)

# Day 1

In [ ]:
import pykx as kx

🌍 Welcome, survivor. You’ve crash-landed in an unknown location. Thankfully, your onboard sensors recorded temperature and humidity readings before the crash. It’s vital to understand the environmental conditions to plan your next steps.

📂 Data: A Parquet file called weather can be found on your laptop. The file contains sensor readings that look like:

| time                    | sensorId | temperatureF | humidity |
|-------------------------|----------|--------------|----------|
| 2026-01-25 00:00:00.000 | 67       | 74.49463     | 62.2822  |
| 2026-01-25 00:00:00.500 | 91       | 73.88359     | 62.53392 |
| 2026-01-25 00:00:01.000 | 42       | 72.1795      | 62.53017 |
| 2026-01-25 00:00:01.500 | 84       | 72.90408     | 64.90296 |
| ...                     |       ...|...           |...       |

🧩 Challenge Using KDB-X, load the Parquet file into a q instance.

Convert the temperatures from Fahrenheit to Celsius.

Calculate the average temperature (in Celsius) for each sensor in daily 1-hour windows (i.e., by date, hour, and sensor).

Calculate the weighted average temperature (in Celsius) for each sensor in daily 1-hour windows using humidity as the weighting factor.

Note: The weather data contains multiple days of sensor readings. The 1 hour windows should be calculated per date and sensorId.

In [ ]:
.pq:use`kx.pq;
t:.pq.pq`:weather.parquet;
tt: select from t;

In [ ]:
temp: update temperatureC: (5%9) * temperatureF-32 from tt;
a:select avg_: avg temperatureC by sensorId, time.date, hour: ("n"$01:00:00) xbar time from temp;
w:select wavg_: humidity wavg temperatureC by sensorId, time.date, hour: ("n"$01:00:00) xbar time from temp;

ans1: first 0!desc select  from a where date=2026.01.25;
ans2: first 0!asc select  from w where date=2026.01.25;

In [ ]:
ans1_1: ans1`sensorId;
ans1_2: ans1`avg_;
ans2_1: ans2`sensorId;
ans2_2: ans2`wavg_;
show (ans1_1; ans1_2; ans2_1; ans2_2);

## Extra exercise

We already know about our environmental conditions, but is it reliable?

We have to discard the unreliable recordings. We know that our sensors only works under strict conditions:
- temperatureC >= -10
- temperatureC * humidity < 3000

To discard the record of a sensor at time t+1 we will use the following approximation

$$temperatureC_{t+1} \simeq temperatureC_t + \Delta temperatureC_t$$

We will suppose the same for the $temperatureC * humidity$ magnitude.

In [ ]:
temp: `time xasc temp;
temp[`valid_record]: 1b;

temp: update valid_record: 0b from temp where -10>temperatureC + temperatureC - prev temperatureC;
temp: update valid_record: 0b from temp where 3000<=(temperatureC * humidity) + deltas temperatureC * humidity;

temp: update valid_record: 0b from temp where -10>temperatureC;
temp: update valid_record: 0b from temp where 3000<=temperatureC * humidity;

reliable_temp: select from temp where valid_record;